In [15]:
# All plots will be displayed directly below the code cell that produced it
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np
import time
import cv2
import operator

# Set inline plots size
plt.rcParams["figure.figsize"] = (64, 40) # (w, h)

In [8]:
!wget "https://drive.google.com/uc?export=view&id=1ho2Wf2RHmeSktY0UD2zf9-kclpnAognG" -O objects.png
clear_output()

In [2]:
# function to read and resize an image
def read_and_resize(filename, grayscale = False, fx= 0.5, fy=0.5):
    if grayscale:
      img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
      imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
      img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    img_result = cv2.resize(img_result, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
    return img_result

    
def showInRow(list_of_images, titles = None, disable_ticks = False):
  count = len(list_of_images)
  for idx in range(count):
    subplot = plt.subplot(1, count, idx+1)
    if titles is not None:
      subplot.set_title(titles[idx])
      
    img = list_of_images[idx]
    cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
    subplot.imshow(img, cmap=cmap)
    if disable_ticks:
      plt.xticks([]), plt.yticks([])
  plt.show()


def generate_colors(num):
  r = lambda: np.random.randint(0,255)
  return [(r(),r(),r()) for _ in range(num)]


def drawContourPoints(arr, image, color):
  for coords in arr:
    cv2.circle(image,(coords[0][0], coords[0][1]), 5, color, -1)

In [9]:
objects = read_and_resize('objects.png')
plt.imshow(objects)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [27]:
def calculate_sizes_and_rank(objects, debug = False):
  #@title Debugging parameters { run: "auto" }
  thresh_val = 170 #@param {type:"slider", min:0, max:255, step:1}

  objects_gray = cv2.cvtColor(objects, cv2.COLOR_BGR2GRAY)

  # Thresholding
  ret,objects_bw = cv2.threshold(objects_gray,thresh_val,255,cv2.THRESH_BINARY)

  kernel_size1 = 25 #@param {type:"slider", min:0, max:50, step:1}
  kernel = np.ones((kernel_size1,kernel_size1),np.uint8)
  edited1 = cv2.morphologyEx(objects_bw, cv2.MORPH_OPEN, kernel)

  kernel_size2 = 33 #@param {type:"slider", min:0, max:50, step:1}
  kernel = np.ones((kernel_size2,kernel_size2),np.uint8)
  edited2 = cv2.erode(edited1,kernel,iterations = 1)

  kernel_size3 = 39 #@param {type:"slider", min:0, max:50, step:1}
  kernel = np.ones((kernel_size3,kernel_size3),np.uint8)
  edited3 = cv2.dilate(edited2,kernel,iterations = 1)

  if debug:
    showInRow([objects_bw, edited1, edited2, edited3], ['binary image', 'edited1', 'edited2', 'edited3'])

  contours, hierarchy = cv2.findContours(edited3, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  im2 = cv2.drawContours(edited3, contours, -1, (255, 0, 0))
  img_contours=objects.copy()

  if debug:
    print('Number of contours: {}'.format(len(contours)))

  # draw contours on input image
  cv2.drawContours(img_contours, contours, -1, (255,0,0), 3)

  if debug:
    plt.subplot(132),plt.imshow(img_contours,cmap = 'gray')
    plt.title('Image with contours'), plt.xticks([]), plt.yticks([])
    
    plt.show()

  image_ = objects.copy()
  colors = generate_colors(len(contours))
  for i in range(len(contours)):
    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(image_, "Area = {}".format(cv2.contourArea(contours[i])), (20, 21*(i+1)), font, 1,colors[i],1, cv2.LINE_AA)
    drawContourPoints(contours[i], image_, colors[i])
    cv2.circle(image_,(10, 20*(i+1)), 7, colors[i], -1)

  if debug:
    plt.imshow(image_,cmap = 'gray')
    plt.show()

  contoursRequired = contours[2:]
  d = dict()
  for i in range(len(contoursRequired)):
    if i == 2:
      d[i] = cv2.contourArea(contoursRequired[i]) - cv2.contourArea(contoursRequired[i+1])
    elif i == 3:
      d[i] = 0
    else:
      d[i] = cv2.contourArea(contoursRequired[i])

  sorted_d = dict( sorted(d.items(), key=operator.itemgetter(1),reverse=True))
  sortedAreas = list(sorted_d.values())

  if debug:
    print(sortedAreas)
    print(sorted_d)

  image2 = objects.copy()
  colors = generate_colors(len(contoursRequired))
  for i in range(len(contoursRequired)):
    if i == 3:
      drawContourPoints(contoursRequired[i], image2, colors[i-1])
    else:
      M = cv2.moments(contoursRequired[i])
      cx = int(M['m10']/M['m00'])
      cy = int(M['m01']/M['m00'])
      cv2.putText(image2, "{}".format(1+sortedAreas.index(sorted_d[i])), (cx, cy), font, 5,(0, 0, 0),3, cv2.LINE_AA)
      font = cv2.FONT_HERSHEY_PLAIN
      if i == 2:
        cv2.putText(image2, "Area = {} pixels".format(cv2.contourArea(contoursRequired[i]) - cv2.contourArea(contoursRequired[i+1])), (20, 21*(i+1)), font, 1,colors[i],1, cv2.LINE_AA)
      else:
        cv2.putText(image2, "Area = {} pixels".format(cv2.contourArea(contoursRequired[i])), (20, 21*(i+1)), font, 1,colors[i],1, cv2.LINE_AA)
      drawContourPoints(contoursRequired[i], image2, colors[i])
      cv2.circle(image2,(10, 20*(i+1)), 7, colors[i], -1)

  plt.imshow(image2,cmap = 'gray')
  plt.show()

In [28]:
#@title Parameters { run: "auto" }
debug = False #@param {type: 'boolean'}
calculate_sizes_and_rank(objects, debug = debug)

Output hidden; open in https://colab.research.google.com to view.